In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It depends on the specific policies of the institution or the course provider. Here are a few steps you can take:\n\n1. **Check the Course Website:** Look for any information about late enrollment or contact details for the instructor or administration.\n \n2. **Contact the Instructor:** Reach out directly to the course instructor to explain your situation and ask if you can still join the course.\n\n3. **Talk to the Administration:** If the course is part of a college or university, get in touch with the registrar's office or the department offering the course. They can let you know if late enrollment is possible and if there are any additional steps you need to take.\n\n4. **Consider Catching Up:** If you are allowed to enroll late, ask if there are recorded lectures or notes available to help you catch up on what you've missed.\n\n5. **Understand the Implications:** Be aware of any consequences or additional requirements for joining late, such as catching up on missed assignments o

In [13]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [26]:
response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": build_prompt(q,search(q))}]
    )
    
response.choices[0].message.content

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homeworks, but keep in mind that there will be deadlines for turning in the final projects, so make sure not to leave everything for the last minute.'

In [27]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [28]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
rag(query)

"To run Kafka, follow these instructions based on the details provided in the FAQ database:\n\n### Java Kafka\nIf you're running a Java Kafka producer/consumer/kstreams, navigate to your project directory and execute the following command in the terminal:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of the JAR file you have built.\n\n### Python Kafka\nIf you're running a Python Kafka producer/consumer, you might need to set up a virtual environment first. Here's how to do it:\n\n1. **Create a virtual environment and install the required packages:**\n\n   ```bash\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n\n2. **Activate the virtual environment (you need to do this every time you work on the project):**\n\n   ```bash\n   source env/bin/activate\n   ```\n\n3. **Deactivate the virtual environment when you're done:**\n\n   

In [18]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework assignments, but be mindful of the deadlines for the final projects. It's advisable not to leave everything until the last minute."

In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
from elasticsearch import Elasticsearch

In [31]:
es_client = Elasticsearch('http://0.0.0.0:9200') 

In [32]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [33]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [34]:
from tqdm.auto import tqdm

In [35]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 40.00it/s]


In [36]:
query = 'I just disovered the course. Can I still join it?'

In [37]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [38]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [39]:
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homeworks. However, please be mindful of the deadlines for turning in the final projects to avoid any last-minute rush.'